# 見て! ルンゲクッタが踊っているよ
# かわいいね

In [1]:
# 微分形
function LJforce(r2::Float64)
    return 12*(1-r2^3)/r2^7
end

function calforce(r,f,N)
  f .=0
  for i in 1:N-1, j in i+1:N
    r2 = (r[i,1]-r[j,1])^2 +(r[i,2]-r[j,2])^2 +(r[i,3]-r[j,3])^2     
    r2 = LJforce(r2)
    
    f[i,1] -= r2 * (r[j,1]-r[i,1]) 
    f[j,1] += r2 * (r[j,1]-r[i,1])
    f[i,2] -= r2 * (r[j,2]-r[i,2]) 
    f[j,2] += r2 * (r[j,2]-r[i,2]) 
    f[i,3] -= r2 * (r[j,3]-r[i,3]) 
    f[j,3] += r2 * (r[j,3]-r[i,3]) 
    
  end
  
end

calforce (generic function with 1 method)

In [2]:
function ルンゲクッタ(r::Array{Float64,2},N::Int64,
    rlist::Array{Float64,3},
    vlist::Array{Float64,3}; 
    T=20., dt=1e-3, repair = 50, Mcutoff=400, gamma = .99)
    

  # はい、ここ盛大にミスってる
  v = r;
  @. v = 0.
  dr1 =r; dr2=r; dr3=r; dr4=r
  dv1 =r; dv2=r; dv3=r; dv4=r
  r1 = r; r2=r; r3=r; r4=r
  v1 = r; v2=r; v3=r; v4=r
  dr = r; dv = v
  
  for (i,t) in enumerate(0:dt:T)
    rlist[:,:,i] = r
    vlist[:,:,i] = v
    
    dr1 = v
    calforce(r,dv1,N)
    
    @. r1 = r + .5dt * dr1
    @. v1 = v + .5dt * dv1
    
    dr2 = v1
    calforce(r1,dv2, N) 

    # k3
    @. r2 = r + .5dt * dr2 
    @. v2 = v + .5dt * dv2
    
    dr3 = v2
    calforce(r2,dv3, N)

    # k4
    @. r3 = r + dt*dr3 
    @. v3 = v + dt*dv3
    
    dr4 = v3
    calforce(r3,dv4, N)

    @. dr = (dt/6.)*(dr1 + 2dr2 + 2dr3 + dr4) 
    @. dv = (dt/6.)*(dv1 + 2dv2 + 2dv3 + dv4)
    @. r += dr    
    @. v += dv 
    
  end
  
end


ルンゲクッタ (generic function with 1 method)

In [3]:
K = 4
L = 5
M = 5
N = K*L*M

r = Array{Float64,2}(undef,N,3)
I = 0
for i in 1:K
  for j in 1:L
    for k in 1:M
      I+=1
      r[I,1] = i*1.
      r[I,2] = j*1.
      r[I,3] = k*1.
  
    end
  end
end

N

100

In [4]:
dt = 1e-2
T = 20

numStep = length(0:dt:T)
rlist = Array{Float64,3}(undef,N,3,numStep)
vlist = Array{Float64,3}(undef,N,3,numStep)

r2 = copy(r)
# 天ぷら 天ぷら
@time ルンゲクッタ(r,N, rlist, vlist;T=T, dt=dt);
# 猪突ウ 猛進!!!
@time ルンゲクッタ(r2,N, rlist, vlist;T=T, dt=dt);

  2.718216 seconds (2.76 M allocations: 128.286 MiB, 3.48% gc time)
  1.933918 seconds (10 allocations: 432 bytes)


In [5]:
rlist[:,:,3]

100×3 Array{Float64,2}:
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
   ⋮          
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN
 NaN  NaN  NaN

# 配列を copy しないので、
# 計算が発散してしまいました
# おれのせいです
# あ～あ